# Web Scraping - Lab

## Introduction

Now that you've seen a more extensive example of developing a web scraping script, it's time to further practice and formalize that knowledge by writing functions to parse specific pieces of information from the web page and then synthesizing these into a larger loop that will iterate over successive web pages in order to build a complete dataset.

## Objectives

You will be able to:

* Navigate HTML documents using Beautiful Soup's children and sibling relations
* Select specific elements from HTML using Beautiful Soup
* Use regular expressions to extract items with a certain pattern within Beautiful Soup
* Determine the pagination scheme of a website and scrape multiple pages

## Lab Overview

This lab will build upon the previous lesson. In the end, you'll look to write a script that will iterate over all of the pages for the demo site and extract the title, price, star rating and availability of each book listed. Building up to that, you'll formalize the concepts from the lesson by writing functions that will extract a list of each of these features for each web page. You'll then combine these functions into the full script which will look something like this:  

```python
df = pd.DataFrame()
for i in range(2,51):
    url = "http://books.toscrape.com/catalogue/page-{}.html".format(i)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    new_titles = retrieve_titles(soup)
    new_star_ratings = retrieve_ratings(soup)
    new_prices = retrieve_prices(soup)
    new_avails = retrieve_avails(soup)
    ...
 ```

In [82]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [83]:
res = requests.get('http://books.toscrape.com/catalogue/page-1.html')

soup = BeautifulSoup(res.content)

In [84]:
# Get list of books

def retrieve_books(soup):
    
    # Locate warning header on the page
    warning_header = soup.find(class_ = 'alert alert-warning')

    # Get the warning header's sibling, where the book names are actually stored
    books_header = warning_header.next_sibling.next_sibling

    # Load all the books on the page
    books = books_header.find_all('li')
    
    return books

## Retrieving Titles

To start, write a function that extracts the titles of the books on a given page. The input for the function should be the `soup` for the HTML of the page.

In [59]:
def retrieve_titles(soup):
    title_list = []
    
    books = retrieve_books(soup)
    
    for book in books:
        try:
            title_list.append(book.find('h3').find('a').attrs['title'])
        except AttributeError:
            continue
    
    return title_list

## Retrieve Ratings

Next, write a similar function to retrieve the star ratings on a given page. Again, the function should take in the `soup` from the given HTML page and return a list of the star ratings for the books. These star ratings should be formatted as integers.

In [62]:
def retrieve_ratings(soup):
    ratings = []
    
    books = retrieve_books(soup)
    
    # Extract the rating and append to the list
    for book in books:
        try:
            ratings.append(book.find('p').attrs['class'][1])
        except AttributeError:
            continue
    
    return ratings

## Retrieve Prices

Now write a function to retrieve the prices on a given page. The function should take in the `soup` from the given page and return a list of prices formatted as floats.

In [65]:
def retrieve_prices(soup):
    prices = []
    
    # Retrieve list of books
    books = retrieve_books(soup)
    
    for book in books:
        try:
            prices.append(float(book.find('p', class_ = 'price_color').text.strip('£')))
        except AttributeError:
            continue
    
    return prices

## Retrieve Availability

Write a function to retrieve whether each book is available or not. The function should take in the `soup` from a given html page and return a list of the availability for each book.

In [72]:
def retrieve_availabilities(soup):
    availability = []
    
    # Retrieve list of books
    books = retrieve_books(soup)
    
    for book in books:
        try:
            availability.append(book.find('p', class_ = 'instock availability').text.strip())
        except AttributeError:
            continue
    
    return availability

## Create a Script to Retrieve All the Books From All 50 Pages

Finally, write a script to retrieve all of the information from all 50 pages of the books.toscrape.com website. 

In [94]:
def flatten_list(l):
    return [item for sub_list in l for item in sub_list]

In [99]:
book_names = []
ratings = []
prices = []
avail = []

for i in range(1,51):
    
    url = 'http://books.toscrape.com/catalogue/page-{}.html'.format(i)
    res = requests.get(url)
    soup = BeautifulSoup(res.content)

    book_names.append(retrieve_titles(soup))
    ratings.append(retrieve_ratings(soup))
    prices.append(retrieve_prices(soup))
    avail.append(retrieve_availabilities(soup))

book_names = flatten_list(book_names)
ratings = flatten_list(ratings)
prices = flatten_list(prices)
avail = flatten_list(avail)

In [103]:
df = pd.DataFrame([book_names,ratings,prices,avail]).T

In [106]:
old_cols = list(range(0,4))
new_cols = ['book_names', 'rating', 'price', 'availability']

column_dict = dict(zip(old_cols,new_cols))

In [107]:
df.rename(columns=column_dict, inplace=True)

In [108]:
df

,book_names,rating,price,availability
0,A Light in the Attic,Three,51.77,In stock
1,Tipping the Velvet,One,53.74,In stock
2,Soumission,One,50.1,In stock
3,Sharp Objects,Four,47.82,In stock
4,Sapiens: A Brief History of Humankind,Five,54.23,In stock
...,...,...,...,...
995,Alice in Wonderland (Alice's Adventures in Won...,One,55.53,In stock
996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",Four,57.06,In stock
997,A Spy's Devotion (The Regency Spies of London #1),Five,16.97,In stock
998,1st to Die (Women's Murder Club #1),One,53.98,In stock


## Level-Up: Write a new version of the script you just wrote. 

If you used URL hacking to generate each successive page URL, instead write a function that retrieves the link from the `"next"` button at the bottom of the page. Conversely, if you already used this approach above, use URL-hacking (arguably the easier of the two methods in this case).

In [ ]:
#Your code here

## Summary

Well done! You just completed your first full web scraping project! You're ready to start harnessing the power of the web!